In [94]:
import random
import pygame
import math
import time


In [95]:
class Flower:
    def __init__(self):
        self.genes = [
            random.randint(1, 10),
            random.randint(0, 255),
            random.randint(0, 255),
            random.randint(0, 255),
            random.randint(0, 255),
            random.randint(0, 255),
            random.randint(0, 255),
            random.randint(0, 255),
            random.randint(0, 255),
            random.randint(0, 255),
            random.randint(0, 7)
        ]

In [96]:
def create_population(n):
    population = []
    for _ in range(n):
        population.append(Flower())
    return population

In [97]:
flowers = create_population(8)
for f in flowers:
    print(f.genes)

[1, 125, 146, 18, 30, 133, 62, 106, 111, 117, 3]
[3, 114, 145, 224, 56, 153, 50, 67, 94, 101, 4]
[4, 95, 33, 241, 217, 215, 16, 35, 56, 219, 5]
[9, 99, 239, 215, 19, 212, 99, 157, 228, 19, 1]
[4, 61, 142, 117, 138, 146, 66, 94, 233, 6, 3]
[9, 243, 175, 212, 152, 115, 76, 246, 87, 164, 1]
[7, 212, 209, 57, 241, 82, 133, 117, 234, 228, 3]
[7, 64, 229, 42, 187, 30, 205, 106, 31, 63, 2]


In [98]:

# Add fitness tracking to flowers
for flower in flowers:
    flower.fitness = 0
    flower.hover_time = 0

def draw_with_hover_tracking(screen, flower, x, mouse_pos):
    size = flower.genes[0]
    center = (flower.genes[1], flower.genes[2], flower.genes[3])
    petalcolor = (flower.genes[4], flower.genes[5], flower.genes[6])
    stem = (flower.genes[7], flower.genes[8], flower.genes[9])
    petals = flower.genes[10]
    
    y = 300 

    # Draw stem
    pygame.draw.line(screen, stem, (x, y + 100), (x, y), 4)

    # Draw petals
    if petals > 0:
        radius = size + 15
        petalsize = 20
        for i in range(petals):
            angle = math.radians(i * (360 / petals))
            px = x + int(math.cos(angle) * radius)
            py = y - int(math.sin(angle) * radius)
            pygame.draw.circle(screen, petalcolor, (px, py), petalsize)

    # Draw center
    pygame.draw.circle(screen, center, (x, y), size)
    
    # Create hover area
    hover_area = pygame.Rect(x - 50, y - 50, 100, 100)
    
    # Check if mouse is hovering over this flower
    is_hovering = hover_area.collidepoint(mouse_pos)
    
    # Draw hover indicator and fitness display
    if is_hovering:
        pygame.draw.rect(screen, (255, 255, 0), hover_area, 3)  # Yellow border when hovering
    
    # Display fitness score above flower
    font = pygame.font.Font(None, 24)
    fitness_text = font.render(f"Fitness: {flower.fitness:.1f}", True, (0, 0, 0))
    screen.blit(fitness_text, (x - 30, y - 80))
    
    return hover_area, is_hovering


In [99]:
import random

def weighted_roulette_selection(flowers):
    """Select a flower using weighted roulette wheel selection based on fitness"""
    fitness_values = [flower.fitness for flower in flowers]
    
    # Handle case where all fitness values are 0 or negative
    min_fitness = min(fitness_values)
    if min_fitness <= 0:
        adjusted_fitness = [f - min_fitness + 0.1 for f in fitness_values]
    else:
        adjusted_fitness = fitness_values
    
    total_fitness = sum(adjusted_fitness)
    
    if total_fitness == 0:
        return random.choice(flowers)
    
    pick = random.uniform(0, total_fitness)
    current = 0
    
    for i, flower in enumerate(flowers):
        current += adjusted_fitness[i]
        if current >= pick:
            return flower
    
    return flowers[-1]

def crossover_flowers(parent1, parent2):
    """Perform crossover between two parent flowers (65% chance)"""
    if random.random() > 0.65:
        return parent1, parent2
    
    offspring1 = Flower()
    offspring2 = Flower()
    
    crossover_point = random.randint(1, len(parent1.genes) - 1)
    
    offspring1.genes = parent1.genes[:crossover_point] + parent2.genes[crossover_point:]
    offspring2.genes = parent2.genes[:crossover_point] + parent1.genes[crossover_point:]
    
    offspring1.fitness = 0
    offspring1.hover_time = 0
    offspring2.fitness = 0
    offspring2.hover_time = 0
    
    return offspring1, offspring2

def mutate_flower(flower):
    """Mutate a flower's genes (5% chance per gene)"""
    for i in range(len(flower.genes)):
        if random.random() < 0.05:
            if i == 0:
                flower.genes[i] = random.randint(1, 10)
            elif i == 10:
                flower.genes[i] = random.randint(0, 7)
            else:
                flower.genes[i] = random.randint(0, 255)
    return flower

def evolve_new_generation():
    """Evolve the flower population to the next generation"""
    global flowers, hover_start_times
    
    print("=== EVOLUTION IN PROGRESS ===")
    for i, flower in enumerate(flowers):
        print(f"Flower {i+1}: Fitness {flower.fitness:.2f}")
    
    new_population = []
    
    while len(new_population) < 8:
        parent1 = weighted_roulette_selection(flowers)
        parent2 = weighted_roulette_selection(flowers)
        
        offspring1, offspring2 = crossover_flowers(parent1, parent2)
        
        offspring1 = mutate_flower(offspring1)
        offspring2 = mutate_flower(offspring2)
        
        new_population.append(offspring1)
        if len(new_population) < 8:
            new_population.append(offspring2)
    
    # Reset fitness for all flowers in new generation
    for flower in new_population:
        flower.fitness = 0
        flower.hover_time = 0
    
    flowers = new_population[:8]
    hover_start_times = [None] * len(flowers)
    
    print("=== NEW GENERATION CREATED ===")
    print("All fitness values reset to 0 for new generation")
    return flowers

print("Evolution functions loaded! Now modify your pygame loop to add the button.")

Evolution functions loaded! Now modify your pygame loop to add the button.


In [ ]:

pygame.init()
pygame.font.init()

WIDTH, HEIGHT = 800, 600
FLOWER_SPACING = WIDTH // 8
FPS = 60 
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Flower Evolution - Hover to Increase Fitness")
clock = pygame.time.Clock()

# Hover tracking variables
hover_start_times = [None] * len(flowers)
FITNESS_RATE = 1.0

# Evolution button
button_rect = pygame.Rect(WIDTH - 200, 10, 180, 40)
button_color = (100, 200, 100)
button_hover_color = (120, 220, 120)
button_text_color = (255, 255, 255)

generation_counter = 1

running = True
while running:
    current_time = time.time()
    mouse_pos = pygame.mouse.get_pos()
    
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        elif event.type == pygame.MOUSEBUTTONDOWN:
            if button_rect.collidepoint(mouse_pos):
                total_fitness = sum(flower.fitness for flower in flowers)
                if total_fitness > 0:
                    evolve_new_generation()
                    generation_counter += 1
                else:
                    print("No flowers have fitness yet! Hover over flowers first.")

    screen.fill((255, 255, 255))

    # Draw flowers and track hovering
    for i, flower in enumerate(flowers):
        x_position = (i * FLOWER_SPACING) + (FLOWER_SPACING // 2)
        hover_area, is_hovering = draw_with_hover_tracking(screen, flower, x_position, mouse_pos)
        
        # Update hover tracking - FIXED LOGIC
        if is_hovering:
            if hover_start_times[i] is None:
                hover_start_times[i] = current_time
        else:
            if hover_start_times[i] is not None:
                session_duration = current_time - hover_start_times[i]
                flower.hover_time += session_duration
                flower.fitness = flower.hover_time * FITNESS_RATE
                hover_start_times[i] = None
    
    # Update fitness for currently hovered flower
    for i, flower in enumerate(flowers):
        if hover_start_times[i] is not None:
            current_session_time = current_time - hover_start_times[i]
            flower.fitness = (flower.hover_time + current_session_time) * FITNESS_RATE
    
    # Draw evolution button
    button_is_hovered = button_rect.collidepoint(mouse_pos)
    current_button_color = button_hover_color if button_is_hovered else button_color
    pygame.draw.rect(screen, current_button_color, button_rect)
    pygame.draw.rect(screen, (0, 0, 0), button_rect, 2)
    
    # Button text
    font = pygame.font.Font(None, 24)
    button_text = font.render("Evolve New Generation", True, button_text_color)
    text_rect = button_text.get_rect(center=button_rect.center)
    screen.blit(button_text, text_rect)
    
    # Display instructions
    font = pygame.font.Font(None, 36)
    instruction_text = font.render("Hover over flowers to increase their fitness!", True, (0, 0, 0))
    screen.blit(instruction_text, (WIDTH // 2 - 250, 50))
    
    # Display generation counter
    gen_text = font.render(f"Generation: {generation_counter}", True, (0, 0, 0))
    screen.blit(gen_text, (10, 10))
    
    # Display best flower
    best_flower = max(flowers, key=lambda f: f.fitness)
    best_text = font.render(f"Best Fitness: {best_flower.fitness:.1f}", True, (0, 100, 0))
    screen.blit(best_text, (WIDTH // 2 - 100, HEIGHT - 50))

    pygame.display.flip()
    clock.tick(FPS)

pygame.quit()


=== EVOLUTION IN PROGRESS ===
Flower 1: Fitness 0.99
Flower 2: Fitness 1.36
Flower 3: Fitness 0.80
Flower 4: Fitness 1.04
Flower 5: Fitness 1.17
Flower 6: Fitness 1.23
Flower 7: Fitness 1.16
Flower 8: Fitness 1.55
=== NEW GENERATION CREATED ===
All fitness values reset to 0 for new generation
=== EVOLUTION IN PROGRESS ===
Flower 1: Fitness 4.17
Flower 2: Fitness 4.17
Flower 3: Fitness 1.24
Flower 4: Fitness 1.05
Flower 5: Fitness 0.92
Flower 6: Fitness 0.85
Flower 7: Fitness 0.66
Flower 8: Fitness 1.33
=== NEW GENERATION CREATED ===
All fitness values reset to 0 for new generation
=== EVOLUTION IN PROGRESS ===
Flower 1: Fitness 1.50
Flower 2: Fitness 1.50
Flower 3: Fitness 1.50
Flower 4: Fitness 0.00
Flower 5: Fitness 0.66
Flower 6: Fitness 0.14
Flower 7: Fitness 0.51
Flower 8: Fitness 0.00
=== NEW GENERATION CREATED ===
All fitness values reset to 0 for new generation
=== EVOLUTION IN PROGRESS ===
Flower 1: Fitness 2.82
Flower 2: Fitness 2.82
Flower 3: Fitness 0.22
Flower 4: Fitness 0